In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import show
import scipy as sp
import glob
import os

In [2]:
path = './SAFeedback1617/2016/'
files = glob.glob(os.path.join(path,'A*.csv'))
df_ea = (pd.read_csv(f).assign(Location = os.path.basename(f).split(' ')[-1].split('.')[0]
                              , Week = os.path.basename(f).split(' ')[2]
                              , Year = 2016) for f in files)
conc_df1 = pd.concat(df_ea, ignore_index=True, sort=True)

In [3]:
conc_df1.head()

,How well are the tutorials paced?,How well is the schedule paced?,How would you rate your overall satisfaction with the Summer Academy this week?,Location,Timestamp,Unnamed: 0,Week,What track are you in?,Year
0,NaN,3,3,Taipei,8/5/2016 1:39:41,NaN,7,NaN,2016
1,NaN,3,4,Taipei,8/5/2016 1:40:47,NaN,7,NaN,2016
2,NaN,3,4,Taipei,8/5/2016 1:40:50,NaN,7,NaN,2016
3,NaN,4,4,Taipei,8/5/2016 1:42:44,NaN,7,NaN,2016
4,NaN,4,5,Taipei,8/5/2016 1:45:13,NaN,7,NaN,2016


In [4]:
conc_df1 = conc_df1.replace('#REF!', np.nan)
conc_df1 = conc_df1.dropna(subset=['How would you rate your overall satisfaction with the Summer Academy this week?'])

In [5]:
conc_df1 = conc_df1.assign(Pacing = conc_df1['How well are the tutorials paced?'].fillna(conc_df1['How well is the schedule paced?']))
conc_df1 = conc_df1.assign(TimeStamp = conc_df1['Timestamp'].fillna(conc_df1['Unnamed: 0']))

In [6]:
conc_df1['Pacing'] = conc_df1['Pacing'].astype('int')
conc_df1['Pacing'] = conc_df1['Pacing'].astype('category')

In [7]:
conc_df1 = conc_df1.rename(index = str, columns = {"How would you rate your overall satisfaction with the Summer Academy this week?": "Rating", "What track are you in?": "Track"})

In [8]:
conc_df1['Rating'] = conc_df1['Rating'].astype('int')

In [9]:
conc_df1['Rating'] = conc_df1['Rating']*2

In [10]:
conc_df1 = conc_df1.drop(['How well are the tutorials paced?', 'How well is the schedule paced?','Timestamp','Unnamed: 0','TimeStamp'], axis = 1)

In [11]:
conc_df1['Pacing'].unique()

[3, 4, 1, 2, 5]
Categories (5, int64): [3, 4, 1, 2, 5]

In [12]:
unique, count = np.unique(conc_df1['Pacing'], return_counts=True)
print(unique, count)

[1 2 3 4 5] [  6  59 571 247  43]


In [13]:
conc_df1['Pacing'] = conc_df1['Pacing'].cat.rename_categories(['Way too slow', 'A little too slow',
                                                              'Just right', 'A little too fast',
                                                              'Way too fast'])

In [14]:
unique, count = np.unique(conc_df1['Pacing'], return_counts=True)
print(unique, count)

['A little too fast' 'A little too slow' 'Just right' 'Way too fast'
 'Way too slow'] [247  59 571  43   6]


In [15]:
conc_df1['Week'] = conc_df1['Week'].astype('int')
conc_df1['Week'] = conc_df1['Week'].astype('category')

In [16]:
unique, count = np.unique(conc_df1['Week'], return_counts=True)
print(unique, count)

[1 2 3 4 5 6 7] [213 161  95 100 130 124 103]


In [17]:
conc_df1['Track'] = conc_df1['Track'].astype('category')

In [18]:
conc_df1.loc[conc_df1['Location'] == 'SG',['Location']] = 'Singapore'
conc_df1['Location'] = conc_df1['Location'].astype('category')
conc_df1['Location'].cat.categories

Index(['HK', 'LA', 'NY', 'SF', 'SV', 'Singapore', 'Taipei', 'Tokyo'], dtype='object')

In [19]:
np.unique(conc_df1['Location'].dropna(), return_counts=True)

(array(['HK', 'LA', 'NY', 'SF', 'SV', 'Singapore', 'Taipei', 'Tokyo'],
       dtype=object), array([  6,  59, 169, 295, 313,  48,  23,  13]))

In [20]:
locdict = {'HK': 'Hong Kong', 'LA': 'Los Angeles', 'NY': 'New York City', 'SF': 'San Francisco'
          , 'SV':'Sunnyvale', 'Singapore': 'Singapore', 'Taipei': 'Taipei', 'Tokyo': 'Tokyo'}

In [21]:
conc_df1['Location'].cat.categories

Index(['HK', 'LA', 'NY', 'SF', 'SV', 'Singapore', 'Taipei', 'Tokyo'], dtype='object')

In [22]:
conc_df1['Location'] = conc_df1['Location'].cat.rename_categories(locdict)

In [23]:
#conc_df1['Location'] = conc_df1['Location'].cat.rename_categories(['Hong Kong','Los Angeles', 'New York City', 'San Francisco', 'Sunnyvale', 'Singapore', 'Taipei', 'Tokyo'])

In [24]:
np.unique(conc_df1['Location'].dropna(), return_counts=True)

(array(['Hong Kong', 'Los Angeles', 'New York City', 'San Francisco',
        'Singapore', 'Sunnyvale', 'Taipei', 'Tokyo'], dtype=object),
 array([  6,  59, 169, 295,  48, 313,  23,  13]))

In [25]:
conc_df1.describe(include='all')

,Rating,Location,Week,Track,Year,Pacing
count,926.00000,926,926.0,808,926.0,926
unique,NaN,8,7.0,4,NaN,5
top,NaN,Sunnyvale,1.0,Apps,NaN,Just right
freq,NaN,313,213.0,395,NaN,571
mean,8.38013,NaN,NaN,NaN,2016.0,NaN
std,1.58445,NaN,NaN,NaN,0.0,NaN
min,2.00000,NaN,NaN,NaN,2016.0,NaN
25%,8.00000,NaN,NaN,NaN,2016.0,NaN
50%,8.00000,NaN,NaN,NaN,2016.0,NaN
75%,10.00000,NaN,NaN,NaN,2016.0,NaN


In [26]:
df2 = pd.read_csv('./SAFeedback1617/2016/Week 8 Feedback (2016, incomplete) - results.csv')
df2.head()

,#,How likely is it that you would recommend the Make School Summer Academy to a friend?,location,track,Start Date (UTC),Submit Date (UTC),Network ID
0,00b836bda84e6bdbe780af97e249e59f,10,New York,summerApps,9/7/16 1:03,9/7/16 1:04,3212b7a834
1,39dde6dc0e1e375845d756fc7e39fc5f,10,San Francisco,summerIntro,9/7/16 1:03,9/7/16 1:04,f4954355aa
2,5e56b9de91670b308cb98dd2848b8739,10,New York,summerIntro,9/7/16 1:03,9/7/16 1:05,3d69ca289b
3,641081d05785b47a0f17448625da0d49,9,Sunnyvale,summerApps (4-week),9/7/16 1:04,9/7/16 1:06,261608f95d
4,c29bdd4f5678d78b450f4494e0f53c8c,3,San Francisco,summerIntro,9/7/16 1:04,9/7/16 1:11,d6672ddf6f


In [27]:
df2 = df2.drop(['#','Start Date (UTC)','Submit Date (UTC)','Network ID'], axis = 1)

In [28]:
df2.loc[df2['location'] == 'New York',['location']] = 'New York City'
df2.loc[:,['track']] = df2['track'].apply(lambda x: x[6:])
df2 = df2.assign(Week = 8, Year = 2016)
df2 = df2.rename(index = str, columns = {"location":"Location", "track":"Track",
                                        "How likely is it that you would recommend the Make School Summer Academy to a friend?": "Rating"})

In [29]:
df2.head()

,Rating,Location,Track,Week,Year
0,10,New York City,Apps,8,2016
1,10,San Francisco,Intro,8,2016
2,10,New York City,Intro,8,2016
3,9,Sunnyvale,Apps (4-week),8,2016
4,3,San Francisco,Intro,8,2016


In [30]:
df2016 = pd.concat((conc_df1, df2), ignore_index=True, sort=True)
df2016.head()

,Location,Pacing,Rating,Track,Week,Year
0,Taipei,Just right,6,NaN,7,2016
1,Taipei,Just right,8,NaN,7,2016
2,Taipei,Just right,8,NaN,7,2016
3,Taipei,A little too fast,8,NaN,7,2016
4,Taipei,A little too fast,10,NaN,7,2016


In [31]:
df2016.dtypes

Location    object
Pacing      object
Rating       int64
Track       object
Week         int64
Year         int64
dtype: object

In [32]:
df2017 = pd.read_csv('./SAFeedback1617/2017/Student Feedback Surveys-Superview.csv')
df2017 = df2017.assign(Year = 2017)
print(df2017.shape)
df2017.head()

(1453, 7)


,ID,Location,Track,Week,Rating (Num),Schedule Pacing,Year
0,134,San Francisco,"Apps, Explorer",Week 1,3,Just right,2017
1,36,Los Angeles,Apps,Week 1,4,A little too fast,2017
2,117,San Francisco,Games,Week 1,4,Way too slow,2017
3,253,NaN,NaN,Week 2,4,A little too fast,2017
4,350,New York City,"Apps, Explorer",Week 1,4,Just right,2017


In [33]:
df2017['Rating (Num)'].unique()

array(['3', '4', '5', '6', '7', '8', '9', '10', '0', '1', '2', '#ERROR!'],
      dtype=object)

In [34]:
df2017 = df2017.replace('#ERROR!', np.nan)
df2017 = df2017.dropna(subset=['Rating (Num)'])
df2017 = df2017.drop(['ID'], axis = 1)

In [35]:
df2017 = df2017.rename(index = str, columns={'Rating (Num)': 'Rating', 'Schedule Pacing': 'Pacing'})

In [36]:
df2017.loc[:,['Week']] = df2017['Week'].apply(lambda x: x[-1]).astype('int')

In [37]:
df2017['Rating'] = df2017['Rating'].astype('int')

In [38]:
df2017.head()

,Location,Track,Week,Rating,Pacing,Year
0,San Francisco,"Apps, Explorer",1,3,Just right,2017
1,Los Angeles,Apps,1,4,A little too fast,2017
2,San Francisco,Games,1,4,Way too slow,2017
3,NaN,NaN,2,4,A little too fast,2017
4,New York City,"Apps, Explorer",1,4,Just right,2017


In [39]:
print(df2017.shape)
df2017.dtypes

(1450, 6)


Location    object
Track       object
Week         int64
Rating       int64
Pacing      object
Year         int64
dtype: object

In [40]:
df2017['Location'].count()

1406

In [41]:
df1617 = pd.concat((df2016, df2017), ignore_index=True, sort=True)

In [42]:
print(df1617.shape)
df1617.head()

(2441, 6)


,Location,Pacing,Rating,Track,Week,Year
0,Taipei,Just right,6,NaN,7,2016
1,Taipei,Just right,8,NaN,7,2016
2,Taipei,Just right,8,NaN,7,2016
3,Taipei,A little too fast,8,NaN,7,2016
4,Taipei,A little too fast,10,NaN,7,2016


In [43]:
df1617['Location'] = df1617['Location'].astype('category')
df1617['Pacing'] = df1617['Pacing'].astype('category')
df1617['Track'] = df1617['Track'].astype('category')
df1617['Year'] = df1617['Year'].astype('category')
df1617['Week'] = df1617['Week'].astype('category')

In [44]:
df1617.dtypes

Location    category
Pacing      category
Rating         int64
Track       category
Week        category
Year        category
dtype: object

In [45]:
df1617.describe(include='all')

,Location,Pacing,Rating,Track,Week,Year
count,2397,2376,2441.000000,2279,2441.0,2441.0
unique,15,5,NaN,8,8.0,2.0
top,San Francisco,Just right,NaN,Apps,1.0,2017.0
freq,684,1605,NaN,1293,501.0,1450.0
mean,NaN,NaN,8.408029,NaN,NaN,NaN
std,NaN,NaN,1.513817,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,8.000000,NaN,NaN,NaN
50%,NaN,NaN,8.000000,NaN,NaN,NaN
75%,NaN,NaN,10.000000,NaN,NaN,NaN


In [46]:
np.unique(df1617['Track'].dropna(), return_counts=True)

(array(['Apps', 'Apps (4-week)', 'Apps, Explorer', 'Games',
        'Games (4-week)', 'Games, Explorer', 'Intro', 'VR'], dtype=object),
 array([1293,    3,  224,  410,    2,   43,  211,   93]))